In [1]:
import sys
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
import importlib
sys.path.append("../")
import metrics
importlib.reload(metrics)

<module 'metrics' from '/Users/narek/Research/library/metrics/metrics/test/../metrics.py'>

In [3]:
%load_ext line_profiler

In [4]:
nC, nI = 1, 1
nL, nLx, nLy, nH = 3, 4, 5, 24

In [5]:
data = np.random.random((100, 10))
Y, X, Z = data[:, :3].T, data[:, 3:7].T, data[:, 7:].T

Ydata = pd.DataFrame(Y.T, index=pd.date_range(start=dt.datetime(2000, 1, 1), periods=100))
Xdata = pd.DataFrame(X.T, index=pd.date_range(start=dt.datetime(2000, 1, 1), periods=100))

In [6]:
metrics.ols_h_njit(Y[0], X)
%timeit metrics.ols_h(Y[0], X)
%timeit metrics.ols_h_njit(Y[0], X)

14.1 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
4.89 µs ± 30.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [7]:
metrics.ols_b_h_njit(Y, X)
%timeit metrics.ols_b_h(Y, X)
%timeit metrics.ols_b_h_njit(Y, X)

65.4 µs ± 23.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
25 µs ± 144 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
metrics.fit_ardl_h_njit(Y[0:1], X, Z, nC, nI, nLy, nLx)
%timeit metrics.fit_ardl_h(Y[0:1], X, Z, nC, nI, nLy, nLx)
%timeit metrics.fit_ardl_h_njit(Y[0:1], X, Z, nC, nI, nLy, nLx)

113 µs ± 41.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
39.1 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
metrics.fit_var_h_njit(Y, nC, nL)
%timeit metrics.fit_var_h(Y, nC, nL)
%timeit metrics.fit_var_h_njit(Y, nC, nL)

99.2 µs ± 43.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
60.5 µs ± 293 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
metrics.lpols_njit(Ydata=Y, Xdata=X, nL=nL, nH=nH)
%timeit metrics.lpols(Ydata=Y, Xdata=X, nL=nL, nH=nH)
%timeit metrics.lpols_njit(Ydata=Y, Xdata=X, nL=nL, nH=nH)

217 µs ± 27.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
88.2 µs ± 3.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Irfs

In [11]:
Mdl = metrics.VARm(Ydata, nL=nL).irf(ci='bs', nH=nH)
Bc, Bx, B, U, S = Mdl.Est['Bc'], Mdl.Est['Bx'], Mdl.Est['B'], Mdl.Est['U'], Mdl.Est['S']
A0inv = np.linalg.inv(S)
Psi = metrics.get_Psi_from_Bx(Bx, nH)
nL, nH = Mdl.Spec['nL'], Mdl.Irfs.Spec['nH']
Y0 = Y[:, :nL]

In [12]:
%timeit metrics.sim_var(Y0, Bc, Bx, U)
%timeit metrics.sim_var_njit(Y0, Bc, Bx, U)

229 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
28.4 µs ± 12.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit metrics.get_Psi_from_Bx(Bx, nH)
%timeit metrics.get_Psi_from_Bx_njit(Bx, nH)

139 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
18.9 µs ± 273 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [14]:
%timeit metrics.get_sirf_from_irf(Bx, A0inv)
%timeit metrics.get_sirf_from_irf_njit(Bx, A0inv)

12.7 µs ± 315 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
The slowest run took 9.63 times longer than the fastest. This could mean that an intermediate result is being cached.
10.6 µs ± 13.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit metrics.get_irfs_VARm(Bx, A0inv, nH)
%timeit metrics.get_irfs_VARm_njit(Bx, A0inv, nH)

28.1 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
25.5 µs ± 198 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Bootstrap vs Wild Bootstrap

In [16]:
%lprun -f metrics.bs_irf Mdl.irf(ci='bs', nH=nH)

Timer unit: 1e-09 s

Total time: 0.017682 s
File: /Users/narek/Research/library/metrics/metrics/test/../metrics.py
Function: bs_irf at line 1025

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1025                                           def bs_irf(Y, U, B, /, *, model_spec, irf_spec, bs_dist = 'Rademacher'):
  1026       100      45000.0    450.0      0.3      nC, nL, nY, nT, dfc = model_spec['nC'], model_spec['nL'], model_spec['nY'], model_spec['nT'], model_spec['dfc']
  1027       100      16000.0    160.0      0.1      nH, ci = irf_spec['nH'], irf_spec['ci']
  1028       100      38000.0    380.0      0.2      Y0 = Y[:, :nL]
  1029       100     958000.0   9580.0      5.4      Bc, Bx = split_C_B(B, nC, nL, nY)
  1030       100      16000.0    160.0      0.1      if ci == 'bs':
  1031       100    1539000.0  15390.0      8.7          idx_r = np.random.choice(nT, size=nT)
  1032       100     472000.0   4720.0      2.7          U_ = U[:, idx_r]
  1033       100   

In [17]:
%lprun -f metrics.bs_irf Mdl.irf(ci='wbs', nH=nH)

Timer unit: 1e-09 s

Total time: 0.016979 s
File: /Users/narek/Research/library/metrics/metrics/test/../metrics.py
Function: bs_irf at line 1025

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1025                                           def bs_irf(Y, U, B, /, *, model_spec, irf_spec, bs_dist = 'Rademacher'):
  1026       100      47000.0    470.0      0.3      nC, nL, nY, nT, dfc = model_spec['nC'], model_spec['nL'], model_spec['nY'], model_spec['nT'], model_spec['dfc']
  1027       100      27000.0    270.0      0.2      nH, ci = irf_spec['nH'], irf_spec['ci']
  1028       100      34000.0    340.0      0.2      Y0 = Y[:, :nL]
  1029       100     923000.0   9230.0      5.4      Bc, Bx = split_C_B(B, nC, nL, nY)
  1030       100      17000.0    170.0      0.1      if ci == 'bs':
  1031                                                   idx_r = np.random.choice(nT, size=nT)
  1032                                                   U_ = U[:, idx_r]
  1033       100   